In [20]:
import requests
from bs4 import BeautifulSoup
import csv
import re
from datetime import datetime  # ✅ Added import for timestamp

API_KEY = "45056db6"
total_movie_titles = []
# ---------------------- #
# 1️⃣ Web Scraping Phase  #
# ---------------------- #
def fetch_movie_titles():
    print("Fetch movie titles from Rotten tomatoes and Box Office Mojo...\n")

    try:
        #Website1: Rotten Tomatoes
        rotten_tomatoes_url = "https://www.rottentomatoes.com/browse/movies_in_theaters"
        #Mimic the browser
        headers = {"User-Agent": "Mozilla/5.0"}
        response_rotten_tomatoes = requests.get(rotten_tomatoes_url, headers = headers)
        #Check the url is working or not
        if response_rotten_tomatoes.status_code != 200:
            print(f"❌failed to fetch the data from Rotten Tomatoes. Status Code: {response_rotten_tomatoes.status_code}")
            return []       
        #Parsing the HTML content from Box Office Mojo
        soup1 = BeautifulSoup(response_rotten_tomatoes.content, "html.parser")
        #Extract movie titles from Rotten Tomatoes website
        rotten_tomatoes_titles = [tag.get("href").split("/")[-1].replace("-", " ").title() 
                                  for tag in soup1.select("a.js-tile-link")]
        print(f"Total ({len(rotten_tomatoes_titles)}) movie titles found in Rotten Tomatos\n")
        ##print(f"✅ Fetched Movie Titles from 🍅 Rotten Tomatoes website...\n")
        print(rotten_tomatoes_titles[:10], "\n") #It is not loading more than 10 movies

        #Website2: Box Office Mojo
        box_office_mojo_url = "https://www.boxofficemojo.com/weekend/"
        response_box_office_mojo = requests.get(box_office_mojo_url, headers = headers)

        #Check the url is working or not
        if response_box_office_mojo.status_code != 200:
            print(f"❌failed to fetch the data from Box Office Mojo. Status Code: {response_box_office_mojo.status_code}")
            return []
        
        #Parsing the HTML content from Box Office Mojo
        soup2 = BeautifulSoup(response_box_office_mojo.text, "html.parser")
        ##Extract movie titles from Box office mojo
        box_office_mojo_titles = [tag.get_text(strip=True)
                              for tag in soup2.select("td.a-text-left.mojo-field-type-release a.a-link-normal")]
        print(f"Total ({len(box_office_mojo_titles)}) movie titles found in Box Office Mojo Website\n")
        #print(soup.prettify())
        print(set(box_office_mojo_titles[:20]),"\n")
        #return box_office_mojo_titles
        total_movie_titles = list(set(rotten_tomatoes_titles + box_office_mojo_titles))
        ##print(f"Total ✅ Fetched movies from Rotten Tomatoes and Box Office Mojo is {total_movie_titles}\n")
        print("Combining the movie titles from Box Office Mojo and Rotten Tomatoes websites....\n")
        print(total_movie_titles,"\n")
        return total_movie_titles

    except requests.exceptions.RequestException as e:
        print(f"❌ Error while scapping Rotten Tomatoes: {e}")
        return[]

# Test the function
#titles = fetch_movie_titles()
print(total_movie_titles)

# ---------------------- #
# 2️⃣ Extract Phase       #
# ---------------------- #
def extract_movies(titles):
    print("🚀 Fetching Data from OMDb API... 🎬")
    movies = []

    for title in titles:
        url = f"http://www.omdbapi.com/?t={title}&apikey={API_KEY}"
        try:
            response = requests.get(url, timeout=10)
            data = response.json()

            if data.get("Response") == "True":
                movies.append(data)
                #print(movies)
            else:
                print(f"❌ Movie not found: {title}")

        except requests.exceptions.RequestException as e:
            print(f"❌ Request Error for {title}: {e}")

    print(f"✅ Fetched {len(movies)} movies successfully!\n")
    return movies

# ---------------------- #
# 3️⃣ Transform Phase      #
# ---------------------- #
def transform_data(movies):
    print("🔄 Transforming Data... 🧹")
    transformed_movies = []  # ✅ Initialize the list

    for movie in movies:
         #TASK1 - Title Transformation ( •	Remove Special Characters: Use regular expressions to eliminate non-alphanumeric characters.)
           #( •	Standardized Case: Convert all titles to Title Case for consistency.)
        #(•	Trim Whitespaces: Remove leading/trailing spaces to maintain uniformity.)

        title_clean = re.sub(r'[^\w\s]', '', movie.get("Title", "").strip()).title() 
        
        #TASK2 - Release Date Transformation (•	Date Formatting: Replace spaces with hyphens to standardize the date format (e.g., DD MMM YYYY to DD-MMM-YYYY).
            #(•	Convert to Date Object: Use date parsing to convert text dates into proper date formats.)
            #(•	Handle Missing Dates: Replace missing dates with "Unknown.")

        release_date = movie.get("Released", "N/A")   
        formatted_date = (
            datetime.strptime(release_date, '%d %b %Y').strftime('%Y-%m-%d') 
            if release_date != "N/A" else "Unknown"
        )
         #TASK3 - Genre Transformation ( •Convert to Lowercase: Ensure all genres are in lowercase for consistency.)
                     #(•Split Genres: If multiple genres are present, separate them into a list.)
                     #(•Remove Duplicates: Ensure unique genre entries.)
        genres = list(set([genre.strip().lower() for genre in movie.get("Genre", "").split(",")])) 

        #TASK4- IMBD Rating Transformation (•Convert to Numeric: Change rating from text to a floating-point number.)
                    #(•	Round Off: Round ratings to one decimal place.)
                    #(•	Normalize: Optionally, normalize ratings on a scale of 0 to 1.)

        imdb_rating = round(float(movie.get("imdbRating", 0)), 1) if movie.get("imdbRating") != "N/A" else None 
        imdb_rating_normalized = imdb_rating / 10 if imdb_rating else None

         #TASK5- Actors Transformation   (•	Limit to Top 3: Display only the top three actors.)
                    #(•	Trim Spaces: Remove extra spaces around names.)
                    #(•	Sort Alphabetically (Optional): For consistency in display.)

        actors = ", ".join(sorted([actor.strip() for actor in movie.get("Actors", "").split(",")[:3]]))

         #TASK6 - BoxOffice Transofrmation   (•	Remove Currency Symbols: Eliminate $, ,, and other non-numeric characters.)
                    #(•	Convert to Numeric: Store as an integer for analysis.)
                    #(•	Handle Missing Data: Replace missing values with 0 or N/A)

        box_office_clean = re.sub(r'[^\d]', '', movie.get("BoxOffice", ""))  
        box_office_value = int(box_office_clean) if box_office_clean else 0

        #TASK7- Awards Transformation    (•	Extract Numbers: Identify and sum all numeric values related to awards won.)
                    #(•	Standardize Format: Display total awards won.)
                    #(•	Handle Missing Awards: Set to 0 if no data is available.)

        awards =movie.get("Awards", "N/A")
        total_awards = 0 #assigning default value
        if awards != "N/A":
            numbers = re.findall(r'\d+', awards)
            total_awards = sum(int(num) for num in numbers)

        #TASK8- MetasScore Transformation  (•Convert to Integer: Change metascore to an integer for calculations.)
                    #(•	Normalize: Convert to a 0-1 scale by dividing by 100.)
                    #(•	Handle Missing Values: Replace "N/A" with None.)

        metascore = movie.get("Metascore", "N/A")
        metascore_normalized = (int(metascore) / 100 if metascore != "N/A" else "None")

        #TASK9 - Language Transformation  (•Convert to Lowercase: Ensure all language names are in lowercase.)
                    #(•	Standardize Codes: Optionally convert to ISO language codes.)
                   #(•	Handle Missing Data: Replace missing languages with "Unknown.")

        language = movie.get("Language", "N/A")
        language = language.lower() if language != "N/A" else "unknown"

        #TASK10 - Production Transformation  (•	Remove Special Characters: Clean the production company names.)
                    #(•	Standardize Names: Correct common misspellings and standardize abbreviations.)
                    #(•	Handle Missing Data: Replace missing production companies with "Independent.")

        production = movie.get("Production", "N/A")
        production = re.sub(r'[^\w\s]', '', production) if production != "N/A" else "Independent"

        transformed_movies.append({
            "Title": title_clean,
            "Release Date": formatted_date,
            "Genres": genres,
            "IMDb Rating": imdb_rating,
            "Normalized IMDb Rating": imdb_rating_normalized,
            "Top 3 Actors": actors,
            "Box Office": box_office_value,
            "Total Awards": total_awards,
            "Metascore": metascore_normalized,
            "Language": language,
            "Production": production
        })

    print(f"✨ Transformation Complete for {len(transformed_movies)} movies!\n")
    return transformed_movies

# ---------------------- #
# 4️⃣ Load Phase (CSV)     #
# ---------------------- #
def load_to_csv(data):
    print("💾 Saving Data to CSV... 📊")
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f"movies_{timestamp}.csv"  # ✅ Add timestamp to avoid overwrite

    with open(filename, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=data[0].keys())
        writer.writeheader()
        writer.writerows(data)

    print(f"✅ Data successfully saved to {filename}!\n")


# ---------------------- #
# 🚀 Execute ETL Pipeline #
# ---------------------- #
if __name__ == "__main__":
    titles = fetch_movie_titles()
    if titles:
        movies = extract_movies(titles)
        if movies:
            transformed_data = transform_data(movies)
            load_to_csv(transformed_data)
            
        else:
            print("⚠️ No movie data fetched from the API.")
    else:
        print("⚠️ No movie titles found during scraping.")


[]
Fetch movie titles from Rotten tomatoes and Box Office Mojo...

Total (11) movie titles found in Rotten Tomatos

['Purple_Rain', 'My_Motherland', 'Open_Your_Eyes_Jeffrey', 'Guns_Of_Redemption', 'The_Visitor_2024', 'The_Way_My_Way', 'Play_It_As_It_Lays', 'The_Corpse', 'A_Match', 'You_Burn_Me'] 

Total (11) movie titles found in Box Office Mojo Website

{'Flight Risk', 'Captain America: Brave New World', 'Den of Thieves: Pantera', 'Mufasa: The Lion King', 'Dog Man'} 

Combining the movie titles from Box Office Mojo and Rotten Tomatoes websites....

['Flight Risk', 'Open_Your_Eyes_Jeffrey', 'The_Visitor_2024', 'A_Match', 'You_Burn_Me', 'Captain America: Brave New World', 'Den of Thieves: Pantera', 'Mufasa: The Lion King', 'Play_It_As_It_Lays', 'Guns_Of_Redemption', 'Ufc_313_Pereira_Vs_Ankalaev', 'Dog Man', 'My_Motherland', 'The_Way_My_Way', 'The_Corpse', 'Purple_Rain'] 

🚀 Fetching Data from OMDb API... 🎬
❌ Movie not found: Open_Your_Eyes_Jeffrey
❌ Movie not found: The_Visitor_2024
❌ M